# JSON Conversions
This notebook demonstrates how to convert a Resource to JSON and vice-versa.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](./00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
#Let get some SHACL shapes from https://github.com/INCF/neuroshapes.git
import os 

neuroshapes_dir = "./neuroshapes"
if os.path.exists(neuroshapes_dir):
  !rm -Rf $neuroshapes_dir
! git clone https://github.com/INCF/neuroshapes.git
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/dataset" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/activity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/entity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/ontology" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/person" "./neuroshapes/shapes/neurosciencegraph/commons/" 

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

## Imports

In [ ]:
import json

In [ ]:
from kgforge.core import Resource

In [ ]:
def pp(x): print(json.dumps(x, indent=4))

## Resource to JSON

In [ ]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [ ]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [ ]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [ ]:
persons = [jane, john]

In [ ]:
forge.register(persons)

In [ ]:
association = Resource(type="Association", agent=persons)

In [ ]:
forge.register(association)

In [ ]:
pp(forge.as_json(association))

In [ ]:
pp(forge.as_json(jane, store_metadata=True))

## JSON to Resource

In [ ]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [ ]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [ ]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [ ]:
persons = [jane, john]

In [ ]:
association = Resource(type="Association", agent=[jane, john])

In [ ]:
data = {
    "type": "Association",
    "agent": [
        {
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva",
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}

In [ ]:
resource = forge.from_json(data)

In [ ]:
resource == association

In [ ]:
print(forge.from_json(data, na="(missing)"))